In [683]:
import json
import requests
import pandas as pd # module to convert an address into latitude and longitude values
import geopandas as gpd
import numpy as num
import folium
from geopy.geocoders import Nominatim #Module for converting the address into latitude and longitude value
from IPython.display import display
from bs4 import BeautifulSoup #Helps us to get the HTML structure of the page

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib.cm import rainbow



In [684]:
Address = '75 Thorncliffe Park Dr, Toronto, Ontario'

geolocation = Nominatim(user_agent="foursquare_agent")
location = geolocation.geocode(Address)
latitude = location.latitude
longitude = location.longitude

query = "Food"
limit = 10
radius = 500
print(latitude, longitude)

43.7049593 -79.3419739


In [685]:
#Call to the api using the stored variables
url = "https://api.foursquare.com/v3/places/search?query={}&ll={},{}&radius={}".format(query,latitude,longitude, radius)

headers = {
    "Accept": "application/json",
    "Authorization": "fsq30W5Rz0P5HJTL4F1jGkXsg37CiGhYOejnEqQCPdKwfUI="
}

#Response from the call
response = requests.request("GET", url, headers=headers)

#Convert the data to JSON
json_data = json.loads(response.text)

#Get the result and store it in list
listt = json_data["results"]

#For every hash, find the key "name"
for restaurant in listt: 
    print(restaurant["name"])


mymap = folium.Map(location=[43.70, -79.34], zoom_start=0)
mymap.save('mymap.html')



Bamiyan Kabob
A&W Restaurant
Pasha Authentic Turkish Cuisine
Faley Restaurant
Afghan Cuisine
Anatolian's Turkish Halal Restaurant
Popeye's Louisiana Kitchen
Hero Certified Burgers
Swiss Chalet Rotisserie & Grill
Capital Sweet Smousa


In [686]:
#Postal Code and Neighborhood Name Data

# reading tables with pandas from url
tables = pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=876823784')


# new dataframe with first table
df = tables[0]
df.head()

# create a new dataframe drop Not assigned in Borough
df1 = df[df['Borough'] != 'Not assigned'].reset_index(drop=True)

postalCodeWithNeighborhoodName = df1.groupby(['Postcode','Borough'], sort=False).agg(', '.join)
postalCodeWithNeighborhoodName.reset_index(inplace=True)

postalCodeWithNeighborhoodName['Neighbourhood'] = num.where(postalCodeWithNeighborhoodName['Neighbourhood'] == 'Not assigned', postalCodeWithNeighborhoodName['Borough'], postalCodeWithNeighborhoodName['Neighbourhood'])

postalCodeWithNeighborhoodName.rename(columns={'Postcode':'Postal Code'}, inplace=True)

postalCodeWithNeighborhoodName.sort_values(['Postal Code'], inplace=True)
postalCodeWithNeighborhoodName = postalCodeWithNeighborhoodName.reset_index(drop=True)

postalCodeWithNeighborhoodName.to_csv("CleanDataSet/PostalCodeWithNeighborhoodName.csv")

postalCodeWithNeighborhoodName


,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [687]:
#Postal Code, Longitude, Latitude Data
postalCodeLongLat = pd.read_csv('PostalCode_Longitute_Latitude.csv')
postalCodeLongLat.rename(columns={'Postal Code':'Postal Code'}, inplace=True)
postalCodeLongLat.to_csv("CleanDataSet/PostalCode_Longitude_Latitude.csv")

postalCodeLongLat.sort_values(['Postal Code'], inplace=True)
postalCodeLongLat = postalCodeLongLat.reset_index(drop=True)

postalCodeLongLat

#postalCodeWithNeighborhoodName.shape

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [688]:
#Join the data only if the postal code key matches
print(postalCodeWithNeighborhoodName.columns)
print(postalCodeLongLat.columns)

#df3 = pd.merge(df2,lat_lng,on='Postcode')
#df3.head(200)

mergeData = pd.merge(postalCodeWithNeighborhoodName, postalCodeLongLat, on='Postal Code')


#Sort the data in terms of postal code (Ascending Order)
mergeData.sort_values(['Postal Code'], inplace=True)
mergeData = mergeData.reset_index(drop=True)
#mergeData.rename(columns={'Postal Code':'Postal Code', 'Postal Code':'Postal Code', 'Neighbourhood': 'Neighbourhood', 'District': 'District', 'Latitude':'Latitude', 'Longitude':'Longitude'}, inplace=True)

mergeData.to_csv('CleanDataSet/MergedPostalCodeandLatitudeLongitude.csv')
mergeData

Index(['Postal Code', 'Borough', 'Neighbourhood'], dtype='object')
Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


In [689]:
#Income Data

#Load CSV File
df = pd.read_csv('neighbourhood-profiles-2016.csv')

#Drop first 3 columns
df.drop(df.columns[[0,1,3]], inplace=True, axis = 1)


df.set_index('Topic', inplace = True)
NeighborhoodIncomes = df.transpose()

NeighborhoodIncomes.reset_index(inplace=True) #reset index of dataframe
for column in NeighborhoodIncomes.columns:
    try:
        column.strip()
    except:
        print(column)

NeighborhoodIncomes.rename(columns={'index': 'Neighbourhood'},inplace = True) #rename column header
NeighborhoodIncomes.drop(NeighborhoodIncomes.columns.difference(['Income of households in 2015', 'Neighbourhood']), axis=1, inplace=True)
NeighborhoodIncomes.columns = NeighborhoodIncomes.iloc[0] #these next 3 lines remove top row
NeighborhoodIncomes = NeighborhoodIncomes.reindex(NeighborhoodIncomes.index.drop(0)).reset_index(drop=True)
NeighborhoodIncomes.columns.name = None 
NeighborhoodIncomes.rename(columns={'Characteristic' : 'Neighbourhood'}, inplace=True)

neighbourhoodCol = NeighborhoodIncomes[['Neighbourhood']].copy()

#Get the column data only
NeighborhoodIncomes = NeighborhoodIncomes.loc[:,'Total - Household total income groups in 2015 for private households - 100% data':'    $200,000 and over']

#Join Neighbourhood Name and all
finalIncome = neighbourhoodCol.join(NeighborhoodIncomes)
finalIncome.sort_values(['Neighbourhood'], inplace=True)
finalIncome = finalIncome.reset_index(drop=True)


finalIncome.to_csv("CleanDataSet/FinalIncome.csv")

finalIncome

,Neighbourhood,Total - Household total income groups in 2015 for private households - 100% data,"Under $5,000","$5,000 to $9,999","$10,000 to $14,999","$20,000 to $24,999","$25,000 to $29,999","$30,000 to $34,999","$35,000 to $39,999","$40,000 to $44,999","$45,000 to $49,999","$50,000 to $59,999","$60,000 to $69,999","$70,000 to $79,999","$80,000 to $89,999","$90,000 to $99,999","$100,000 and over","$200,000 and over"
0,Agincourt North,"9,120",155,105,160,320,540,420,455,420,435,800,700,635,525,515,"2,505",325
1,Agincourt South-Malvern West,"8,135",315,140,195,315,400,370,385,370,415,770,645,595,510,405,"2,030",285
2,Alderwood,"4,620",55,45,80,145,150,155,170,160,165,335,300,320,275,250,"1,915",360
3,Annex,"15,935",850,485,655,620,530,525,555,540,505,"1,000",900,795,715,605,"5,895","2,670"
4,Banbury-Don Mills,"12,125",265,155,235,395,445,405,470,460,400,930,885,780,655,605,"4,615","1,750"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,Wychwood,"5,885",120,120,185,325,270,225,225,225,225,400,390,315,300,275,"1,895",610
137,Yonge-Eglinton,"5,680",205,105,145,175,170,190,210,195,180,370,375,325,275,245,"2,340","1,110"
138,Yonge-St.Clair,"7,015",215,120,185,225,220,215,235,260,260,505,445,415,340,325,"2,865","1,380"
139,York University Heights,"10,165",345,230,340,525,525,590,570,530,530,950,845,615,605,515,"1,955",195


In [690]:
#Crime Rate Data

crimeRateData = pd.read_csv('Neighbourhood_Crime_Rates_(Boundary_File)_.csv')
crimeRateData.drop(crimeRateData.columns[[0]], inplace=True, axis = 1)
crimeRateData.rename(columns={'Neighbourhood':'Neighbourhood'}, inplace=True)
crimeRateData.head()

#neighbourhoodInformation = pd.DataFrame(getCoordinates,columns=['FIELD_7', 'FIELD_12', 'FIELD_11'])

finalCrimeRateData = pd.DataFrame(crimeRateData,columns=['Neighbourhood', 'Assault_AVG', 'AutoTheft_AVG', 'BreakandEnter_AVG', 'Homicide_AVG', 'Robbery_AVG', 'TheftOver_AVG'])

finalCrimeRateData.sort_values(['Neighbourhood'], inplace=True)
finalCrimeRateData = finalCrimeRateData.reset_index(drop=True)

finalCrimeRateData.to_csv("CleanDataSet/CrimeRateData.csv")
finalCrimeRateData



,Neighbourhood,Assault_AVG,AutoTheft_AVG,BreakandEnter_AVG,Homicide_AVG,Robbery_AVG,TheftOver_AVG
0,Agincourt North,74.8,29.7,53.5,0.2,30.2,4.7
1,Agincourt South-Malvern West,117.8,36.7,79.8,0.2,27.3,13.3
2,Alderwood,36.3,16.2,24.7,0.2,6.8,6.8
3,Annex,246.3,22.0,147.5,0.5,40.8,29.5
4,Banbury-Don Mills,80.5,21.8,73.2,0.0,15.0,10.3
...,...,...,...,...,...,...,...
135,Wychwood,70.2,13.2,34.0,0.3,13.8,2.3
136,Yonge-Eglinton,75.8,9.0,28.0,0.3,19.5,4.8
137,Yonge-St.Clair,31.0,4.3,23.3,0.0,5.7,4.3
138,York University Heights,333.2,106.3,113.2,0.8,75.8,36.3


In [706]:
#Get Latitude and Longitude from Neighbourhoods File

#Read the shape file
getCoordinates = gpd.read_file("Neighbourhoods/Neighbourhoods.shp")

#Convert to Coordinate Reference System to get Latitdue and Longitude
getCoordinates = getCoordinates.to_crs(epsg=4326)

getCoordinates.head()

#Enter data into a dataframe
neighbourhoodInformation = pd.DataFrame(getCoordinates,columns=['FIELD_7', 'FIELD_12', 'FIELD_11'])

#Rename all columns
neighbourhoodInformation.rename(columns={'FIELD_7':'Neighbourhood', 'FIELD_12':'Latitude', 'FIELD_11':'Longitude'}, inplace=True)

#Get all the values before the bracket
neighbourhoodInformation['Neighbourhood'] = neighbourhoodInformation["Neighbourhood"].astype(str)
neighbourhoodInformation['Neighbourhood'] = neighbourhoodInformation['Neighbourhood'].str.split(" \(", 1)
neighbourhoodInformation['Neighbourhood'] = neighbourhoodInformation['Neighbourhood'].str.get(0)

#Sort all the data by Neighbourhood Name and reset index
neighbourhoodInformation.sort_values(['Neighbourhood'], inplace=True)
neighbourhoodInformation = neighbourhoodInformation.reset_index(drop=True)

neighbourhoodInformation.to_csv("CleanDataSet/NeighbourhoodInformation.csv")
neighbourhoodInformation



,Neighbourhood,Latitude,Longitude
0,Agincourt North,43.805441,-79.266712
1,Agincourt South-Malvern West,43.788658,-79.265612
2,Alderwood,43.604937,-79.541611
3,Annex,43.671585,-79.404001
4,Banbury-Don Mills,43.737657,-79.349718
...,...,...,...
135,Wychwood,43.676919,-79.425515
136,Yonge-Eglinton,43.704689,-79.403590
137,Yonge-St.Clair,43.687859,-79.397871
138,York University Heights,43.765736,-79.488883


In [707]:
combinedTopics = neighbourhoodInformation.merge(finalCrimeRateData, on='Neighbourhood').merge(finalIncome, on='Neighbourhood')

combinedTopics.sort_values(['Neighbourhood'], inplace=True)
combinedTopics = combinedTopics.reset_index(drop=True)

combinedTopics

,Neighbourhood,Latitude,Longitude,Assault_AVG,AutoTheft_AVG,BreakandEnter_AVG,Homicide_AVG,Robbery_AVG,TheftOver_AVG,Total - Household total income groups in 2015 for private households - 100% data,...,"$35,000 to $39,999","$40,000 to $44,999","$45,000 to $49,999","$50,000 to $59,999","$60,000 to $69,999","$70,000 to $79,999","$80,000 to $89,999","$90,000 to $99,999","$100,000 and over","$200,000 and over"
0,Agincourt North,43.805441,-79.266712,74.8,29.7,53.5,0.2,30.2,4.7,"9,120",...,455,420,435,800,700,635,525,515,"2,505",325
1,Agincourt South-Malvern West,43.788658,-79.265612,117.8,36.7,79.8,0.2,27.3,13.3,"8,135",...,385,370,415,770,645,595,510,405,"2,030",285
2,Alderwood,43.604937,-79.541611,36.3,16.2,24.7,0.2,6.8,6.8,"4,620",...,170,160,165,335,300,320,275,250,"1,915",360
3,Annex,43.671585,-79.404001,246.3,22.0,147.5,0.5,40.8,29.5,"15,935",...,555,540,505,"1,000",900,795,715,605,"5,895","2,670"
4,Banbury-Don Mills,43.737657,-79.349718,80.5,21.8,73.2,0.0,15.0,10.3,"12,125",...,470,460,400,930,885,780,655,605,"4,615","1,750"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,Wychwood,43.676919,-79.425515,70.2,13.2,34.0,0.3,13.8,2.3,"5,885",...,225,225,225,400,390,315,300,275,"1,895",610
132,Yonge-Eglinton,43.704689,-79.403590,75.8,9.0,28.0,0.3,19.5,4.8,"5,680",...,210,195,180,370,375,325,275,245,"2,340","1,110"
133,Yonge-St.Clair,43.687859,-79.397871,31.0,4.3,23.3,0.0,5.7,4.3,"7,015",...,235,260,260,505,445,415,340,325,"2,865","1,380"
134,York University Heights,43.765736,-79.488883,333.2,106.3,113.2,0.8,75.8,36.3,"10,165",...,570,530,530,950,845,615,605,515,"1,955",195


In [744]:
print(combinedTopics.columns)
print(mergeData.columns)



#finalDataSet = combinedTopics.merge(mergeData)

Index(['Neighbourhood', 'Latitude', 'Longitude', 'Assault_AVG',
       'AutoTheft_AVG', 'BreakandEnter_AVG', 'Homicide_AVG', 'Robbery_AVG',
       'TheftOver_AVG',
       'Total - Household total income groups in 2015 for private households - 100% data',
       '  Under $5,000', '  $5,000 to $9,999', '  $10,000 to $14,999',
       '  $20,000 to $24,999', '  $25,000 to $29,999', '  $30,000 to $34,999',
       '  $35,000 to $39,999', '  $40,000 to $44,999', '  $45,000 to $49,999',
       '  $50,000 to $59,999', '  $60,000 to $69,999', '  $70,000 to $79,999',
       '  $80,000 to $89,999', '  $90,000 to $99,999', '  $100,000 and over',
       '    $200,000 and over', 'geometry'],
      dtype='object')
Index(['Postal Code', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude',
       'geometry'],
      dtype='object')


,Neighbourhood_left,Latitude_left,Longitude_left,Assault_AVG,AutoTheft_AVG,BreakandEnter_AVG,Homicide_AVG,Robbery_AVG,TheftOver_AVG,Total - Household total income groups in 2015 for private households - 100% data,...,"$90,000 to $99,999","$100,000 and over","$200,000 and over",geometry,index_right,Postal Code,Borough,Neighbourhood_right,Latitude_right,Longitude_right
0,Agincourt North,43.805441,-79.266712,74.8,29.7,53.5,0.2,30.2,4.7,"9,120",...,515,"2,505",325,POINT (43.80544 -79.26671),NaN,NaN,NaN,NaN,NaN,NaN
1,Agincourt South-Malvern West,43.788658,-79.265612,117.8,36.7,79.8,0.2,27.3,13.3,"8,135",...,405,"2,030",285,POINT (43.78866 -79.26561),NaN,NaN,NaN,NaN,NaN,NaN
2,Alderwood,43.604937,-79.541611,36.3,16.2,24.7,0.2,6.8,6.8,"4,620",...,250,"1,915",360,POINT (43.60494 -79.54161),NaN,NaN,NaN,NaN,NaN,NaN
3,Annex,43.671585,-79.404001,246.3,22.0,147.5,0.5,40.8,29.5,"15,935",...,605,"5,895","2,670",POINT (43.67159 -79.40400),NaN,NaN,NaN,NaN,NaN,NaN
4,Banbury-Don Mills,43.737657,-79.349718,80.5,21.8,73.2,0.0,15.0,10.3,"12,125",...,605,"4,615","1,750",POINT (43.73766 -79.34972),NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,Wychwood,43.676919,-79.425515,70.2,13.2,34.0,0.3,13.8,2.3,"5,885",...,275,"1,895",610,POINT (43.67692 -79.42551),NaN,NaN,NaN,NaN,NaN,NaN
132,Yonge-Eglinton,43.704689,-79.403590,75.8,9.0,28.0,0.3,19.5,4.8,"5,680",...,245,"2,340","1,110",POINT (43.70469 -79.40359),NaN,NaN,NaN,NaN,NaN,NaN
133,Yonge-St.Clair,43.687859,-79.397871,31.0,4.3,23.3,0.0,5.7,4.3,"7,015",...,325,"2,865","1,380",POINT (43.68786 -79.39787),NaN,NaN,NaN,NaN,NaN,NaN
134,York University Heights,43.765736,-79.488883,333.2,106.3,113.2,0.8,75.8,36.3,"10,165",...,515,"1,955",195,POINT (43.76574 -79.48888),NaN,NaN,NaN,NaN,NaN,NaN


In [745]:
address = 'Agincourt North'
geolocator = Nominatim(user_agent = 'ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.8080383, -79.2664391.


In [746]:
#Call to the api using the stored variables
url = "https://api.foursquare.com/v3/places/search?query={}&ll={},{}&radius={}".format(query,latitude,longitude, radius)

headers = {
    "Accept": "application/json",
    "Authorization": "fsq30W5Rz0P5HJTL4F1jGkXsg37CiGhYOejnEqQCPdKwfUI="
}

#Response from the call
response = requests.request("GET", url, headers=headers)

#Convert the data to JSON
json_data = json.loads(response.text)

mymap = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map

for lat, lang, neighborhood in zip(neighbourhoodInformation['Latitude'], neighbourhoodInformation['Longitude'], neighbourhoodInformation['Neighbourhood']):
    label = '{}. {}. {}'.format(neighborhood, lat, lang)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lang],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(mymap)  

mymap